# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [5]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

In [9]:
data = pd.read_excel('../data/Online Retail.xlsx')

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [10]:
april = data[(data['InvoiceDate'] >= '2011-04-01') & 
             (data['InvoiceDate'] < '2011-05-01') & 
             (data['Country'] != 'United Kingdom')]

grouped = april.groupby('Country').agg('sum')[['Quantity', 'Revenue']]

grouped.iplot(kind='bar', xTitle='Country', yTitle='Amount', 
              title='April 2011 Quantity & Revenue by Country (Excl. UK)')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [12]:
france = data[(data['InvoiceDate'] >= '2011-01-01') & 
              (data['InvoiceDate'] < '2011-06-01') & 
              (data['Country'] == 'France')]

grouped = france.groupby('InvoiceDate').agg('sum')[['Quantity', 'Revenue']]
grouped.iplot(kind='line', title='France Quantity & Revenue Over Time')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [13]:
bunting = data[data['Description']=='PARTY BUNTING'][['Country', 'Quantity', 'UnitPrice']]
grouped = bunting.groupby('Country').agg('mean').reset_index()

grouped.iplot(kind='scatter', x='Quantity', y='UnitPrice', categories='Country',
              xTitle='Avg. Quantity', yTitle='Avg. Unit Price', 
              title='Avg. Quantity vs. Unit Price by Country')

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [14]:
countries = data[data['Country'].isin(['EIRE', 'Germany', 'France', 'Netherlands'])]
invoices = countries.pivot_table(values='Quantity', columns='Country', index='InvoiceNo', aggfunc='mean')

invoices.iplot(kind='hist', subplots=True, subplot_titles=True, legend=False,
               title='Sales Quantity Distributions by Country')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [15]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [16]:
products = data[(data['Description'].isin(product_list)) & 
                (data['Country'].isin(country_list))]

pivot = products.pivot_table(values='Revenue', columns='Country', index='Description', aggfunc='mean')

pivot.iplot(kind='bar', xTitle='Product', yTitle='Revenue',
            title='Product Revenue by Country')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [17]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [18]:
@interact(Year=uk['Year'].unique(), 
          Month=uk['Month'].unique())

def linechart(Year, Month):
    df = uk[(uk['Year']==Year) & (uk['Month']==Month)]
    grouped = df.groupby('Day').agg('sum')['Quantity']
    grouped.iplot(kind='line', title='UK Sales Quantity by Day')

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [19]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [20]:
@interact(Customers = (products['CustomerID'].min(), products['CustomerID'].max()),
          Invoices = (products['InvoiceNo'].min(), products['InvoiceNo'].max()))

def scatter(Customers, Invoices):
    df = products[(products['CustomerID'] > Customers) & 
                  (products['InvoiceNo'] > Invoices)].reset_index()
    
    df.iplot(kind='scatter', x='InvoiceNo', y='CustomerID', mode='markers', 
             text='Description', xTitle='Invoices', yTitle='Customers', 
             title='Number of Invoices vs. Customers by Product')

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [21]:
@interact(Product='')

def chart(Product):
    df = data[data['Description'].str.contains(Product.upper())]
    grouped = df.groupby('Description').agg({'Revenue':'sum'})
    grouped.iplot(kind='bar')